# Création des resources RDF des mentions de parcelles

In [339]:
import json 
import pandas as pd
import numpy as np
import uuid
import re
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import XSD, DCTERMS, PROV, SKOS, RDFS
from functions import *

## 1. Lecture des données

### Articles de classement

In [340]:
COMMUNE = 'Gentilly'
matrices_metada = {
    "MAT_1813": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_B_NB_1813",
        "MATRICE_START": "1813",
        "MATRICE_END": "1835"
    },
    "MAT_1836": {
        "PLAN": "1811",
        "MATRICE_ID": "MAT_NB_1836",
        "MATRICE_START": "1836",
        "MATRICE_END": "1847"
    },
    "MAT_1848": {
        "PLAN": "1845",
        "MATRICE_ID": "MAT_NB_1848",
        "MATRICE_START": "1848",
        "MATRICE_END": "1860"
    }
}


In [341]:
ROOT = "/workspaces/ontologie-peuplement/"  #/home/STual/KG-cadastre/

PATH = ROOT + "data/gentilly/MAT_1813.csv"
mat1813 = pd.read_csv(PATH,header=0)
PATH = ROOT + "data/gentilly/MAT_1836.csv"
mat1836 = pd.read_csv(PATH,header=0)
PATH = ROOT + "data/gentilly/MAT_1848.csv"
mat1848 = pd.read_csv(PATH,header=0)

OUTPUT_FOLDER_PATH = ROOT + "data/rdf"

In [342]:
mat1813['registre'] = 'MAT_1813'
mat1836['registre'] = 'MAT_1836'
mat1848['registre'] = 'MAT_1848'

matrices = pd.concat([mat1813, mat1836, mat1848])
matrices = matrices.reset_index(drop=True)
print(matrices.columns)

Index(['ID', 'UUID', 'Type_CF', 'Num_Folio', 'Alt_Num_CF', 'Groupe_CF',
       'Ordre_de_lecture', 'Voie', 'Num_Voie', 'Image', 'Section_clean',
       'Parcelle_clean', 'Lieu-dit_transcript', 'Lieu-dit_clean',
       'Lieu-dit_treated', 'Lieu-dit_type', 'Propriétaires_transcript',
       'Nature_transcript', 'Nature_clean', 'Nature_treated', 'Date entrée',
       'Date entrée_treated', 'Date sortie', 'Date sortie_treated', 'Tiré de',
       'Tiré de_treated', 'Porté à', 'Porté à_treated', 'Ligne barrée ?',
       'CF rayé ?', 'Spécification', 'Commentaire', 'Cote liée', 'registre'],
      dtype='object')


In [343]:
len(matrices)

442

### Propriétaires

In [344]:
#open three json files
with open(ROOT + "data/gentilly/output_structured_owners.json") as f:
    data_owners = json.load(f)

#read as df
owners_df = pd.DataFrame(data_owners)

In [345]:
oregistre = []
otype_folio = []
ofolio = []
o_groupe_cf = []
o_transcription = []

for row in owners_df.iterrows():
    cell_info = row[1]['cell']
    oregistre.append(cell_info['registre'])
    otype_folio.append(cell_info['type_folio'])
    ofolio.append(cell_info['folio'])
    o_groupe_cf.append(cell_info['groupe_cf'])
    o_transcription.append(cell_info['transcription'])

owners_df['registre'] = oregistre
owners_df['type_folio'] = otype_folio
owners_df['folio'] = ofolio
owners_df['groupe_cf'] = o_groupe_cf
owners_df['transcription'] = o_transcription


In [346]:
owners_df

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],MAT_1813,Bâti,108,1,Legendre H↑re↓ de →Fontainebleau
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris"
3,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Faipot', '...",[],MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...
4,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Bacot', 'o...",[],MAT_1813,Non Bâti,11,1,Bacot→(david) couverturier→à Paris
...,...,...,...,...,...,...,...,...
185,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Delon', 'i...","[{'change-order': 1, 'owner-before': 1, 'owner...",MAT_1848,Non Bâti,1218,1,~~Delon~~ 1858 Sevestre Clément boulanger Bar↑...
186,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Delon', 'i...",[],MAT_1848,Non Bâti,1219,1,Delon
187,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Dufresne',...",[],MAT_1848,Non Bâti,1220,1,Dufresne
188,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul


### 1.1. Création des pages
- rdf:type rico:Instanciation : instance numérisée d'une page de registre
- rdf:type rico:Record => concept de la page de registre, fait le lien avec le registre (concept, RecordSet)

In [347]:
#select distinct values in th colum Image
images = matrices[['registre','Image']].drop_duplicates().reset_index(drop=True)
images

,registre,Image
0,MAT_1813,FRAD094_3P_000255_01_0586
1,MAT_1813,FRAD094_3P_000255_01_0588
2,MAT_1813,FRAD094_3P_000255_01_0579
3,MAT_1813,FRAD094_3P_000255_01_0571
4,MAT_1813,FRAD094_3P_000255_01_0015
...,...,...
150,MAT_1848,FRAD094_3P_000264_01_0218
151,MAT_1848,FRAD094_3P_000264_01_0219
152,MAT_1848,FRAD094_3P_000264_01_0220
153,MAT_1848,FRAD094_3P_000264_01_0221


In [348]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('cad', cad)
g.bind('add', add)
g.bind('source', srcuri)
g.bind('mlclasse', mlclasse)
g.bind('activity', cad_act)

g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

for index, row in images.iterrows():
    img = row['Image']
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]
    json = parse_record_id(img)
    subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{img}")
    g.add((subject_uri, RDF.type, rico.Instanciation))
    g.add((subject_uri, rico.identifier, Literal(img)))
    mlClasseNode = BNode()
    g.add((subject_uri, cad.hasClasse, mlClasseNode))
    g.add((mlClasseNode, cad.hasClasseValue, URIRef(mlclasse + f"MATMainTable")))
    g.add((mlClasseNode, PROV.wasGeneratedBy, URIRef(cad_act + f"0001")))
    folder_end = img.rfind('_')
    g.add((subject_uri,rico.isOrWasDigitalInstanciationOf,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{img}_page")))

    subject_uri_record = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")
    g.add((subject_uri_record, RDF.type, rico.Record))
    g.add((subject_uri_record, rico.isOrWasIncludedIn, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))

print(g.serialize(format='turtle'))
#write g into a .ttl file
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_sources_pages.ttl", format='turtle')

@prefix activity: <http://data.ign.fr/id/codes/cadastre/activity/> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix mlclasse: <http://data.ign.fr/id/codes/cadastre/mlClasse/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .

source:94_Gentilly_FRAD094_3P_000255_01_0015 a rico:Instanciation ;
    cad:hasClasse [ cad:hasClasseValue mlclasse:MATMainTable ;
            prov:wasGeneratedBy activity:0001 ] ;
    rico:identifier "FRAD094_3P_000255_01_0015" ;
    rico:isOrWasDigitalInstanciationOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:94_Gentilly_FRAD094_3P_000255_01_0032 a rico:Instanciation ;
    cad:hasClasse [ cad:hasClasseValue mlclasse:MATMainTable ;
            prov:wasGeneratedBy activity:0001 ] ;
    rico:identifier "FRAD094_3P_000255_01_0032" ;
    rico:isOrWasDigitalInstanciationOf source:94_Gentilly_FRAD094_3P_000255_01_0032_page .

source:9

<Graph identifier=Nfff1de836a304fdda2284561b6d29092 (<class 'rdflib.graph.Graph'>)>

### 1.2 Folios
- Pré-traitement des colonnes *Num_Folio*, *Tiré de* et *Porté à*
- Création des objets "Folios" à partir de la colonne *Num_Folio* et des colonnes *Tiré de* et *Porté à* (manquants)
- Création des objets spéciaux mentionnés dans les colonnes destinées aux folios (reste, construction nouvelle, ruine etc)

#### Pré-traitement

In [349]:
from functions import parse_record_id, cleanNumFolio

#Clean columns Num_Folio, Tire_de, Porte_a
clean_folio, clean_tire_de, clean_porte_a = [], [], []
symbols = [",", "→", "."," ",";","&"]

for index, row in matrices.iterrows():
    clean_folio.append(cleanNumFolio(row["Num_Folio"],symbols))
    clean_tire_de.append(cleanNumFolio(row["Tiré de_treated"],symbols))
    clean_porte_a.append(cleanNumFolio(row["Porté à_treated"],symbols))

# Create new columns containing the cleaned values
matrices['Num_Folio_clean'] = clean_folio
matrices['Tire_de_clean'] = clean_tire_de
matrices['Porte_a_clean'] = clean_porte_a

matrices['Num_Folio_clean'] = matrices['Num_Folio_clean'].astype(str)

['236↑4↓', '361', '258', '166', '235↑2↓', '138', '357', '440']
['248', 'additionconstructionsv']
['249', '249']
['288', '433']
['443', '443']
['450', '443', '453', '968']
['449', '968']
['968', '442']
['442', '449', '439', '450']
['836', '403']
['837', '403']
['403', '513↑25↓']
['837', 'additionconstructionsv']
['844', 'additionconstructionsv']
['844', 'additionconstructionsv']
['846↑2↓', 'additionconstructionsv']
['847↑2↓', 'additionconstructionsv']
['847↑2↓', 'additionconstructionsv']


In [350]:
print(clean_porte_a)

['EMPTY', '82', 'EMPTY', 'EMPTY', 'EMPTY', '156', '156', '192bis', '280bis', 'EMPTY', 'EMPTY', '34ter', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '124bis', 'EMPTY', 'EMPTY', '192bis', '247ter', 'voiepubliquesv', 'EMPTY', 'EMPTY', '46bis', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '211bis', '107↑2↓', 'EMPTY', 'doubleemploisv', 'EMPTY', 'EMPTY', '269↑2↓', '138', '236ter', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '361', 'EMPTY', 'EMPTY', 'EMPTY', '236↑4↓;361;258;166;235↑2↓;138;357;440', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', '249', 'EMPTY', 'EMPTY', '248', 'EMPTY', '249;249', 'EMPTY', 'EMPTY', '249↑16↓', '249↑16↓', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'EMPTY', 'demolitionsv', 'EMPTY', 'EMPTY', 'demolitionsv', '288;433', 'EMPTY', 'EMPTY', 'EMPTY', 'augmentationsv', 'EMPTY', 'EMPTY', 'EMPTY

In [351]:
#Using matrices, create new df named folios containing all lines of matrices where register = MAT_1836 and MAT_1848. For register=MAT_1813, remove the lines where type_CF = "Bâti"
folios = matrices[(matrices['registre'] == 'MAT_1836') | (matrices['registre'] == 'MAT_1848') | ((matrices['registre'] == 'MAT_1813') & (matrices['Type_CF'] != 'Bâti'))]
folios.reset_index(drop=True)

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe_CF,Ordre_de_lecture,Voie,Num_Voie,Image,...,Porté à_treated,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée,registre,Num_Folio_clean,Tire_de_clean,Porte_a_clean
0,6,7008c4af-2b07-4630-8667-9c8783aced10,Non bâti,11,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0015,...,156,Non,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,156
1,7,ef348452-7650-41a5-ac99-61d2490800b5,Non bâti,11,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0015,...,156,Non,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,156
2,8,6e7b862a-4f66-461c-96d4-db9f2dc76e56,Non bâti,11,NaN,1,3,NaN,NaN,FRAD094_3P_000255_01_0015,...,192bis,Oui,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,192bis
3,9,96e406d5-0647-4889-954d-b821f15ba128,Non bâti,23bis,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0032,...,280bis,Oui,Oui,NaN,NaN,NaN,MAT_1813,23bis,constructionnouvellesv,280bis
4,10,3076b547-3147-4247-b503-74e9ead7af45,Non bâti,34ter,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0044,...,NaN,Non,Oui,NaN,NaN,NaN,MAT_1813,34ter,46ter,EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,219,da62bcf1-3993-420b-8aac-e7cd1d1b4cda,Non Bâti,1221,NaN,1,6,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1221,EMPTY,EMPTY
433,220,d0f7ae85-fd51-459b-a4b4-48a8335cf44a,Non Bâti,1221,NaN,1,7,NaN,NaN,FRAD094_3P_000264_01_0221,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1221,EMPTY,EMPTY
434,221,e0d84f8b-4b9b-4b54-92f0-e1d36d5e6e45,Non Bâti,1222,NaN,1,1,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1222,EMPTY,EMPTY
435,222,af70cad2-55f9-4163-8c6b-ece562802b66,Non Bâti,1222,NaN,1,2,NaN,NaN,FRAD094_3P_000264_01_0222,...,NaN,Non,Non,NaN,NaN,NaN,MAT_1848,1222,EMPTY,EMPTY


Il doit rester 437 lignes sur 442.

In [352]:
#Create a new dataframe from sources with columns "Num_Folio" and "Image" containing only distinct rows
folios_pages = folios[["Num_Folio","Num_Folio_clean","Alt_Num_CF","Image","registre"]].drop_duplicates(subset=["Num_Folio","Num_Folio_clean","Alt_Num_CF","Image","registre"]).reset_index(drop=True)
display(folios_pages)

,Num_Folio,Num_Folio_clean,Alt_Num_CF,Image,registre
0,11,11,NaN,FRAD094_3P_000255_01_0015,MAT_1813
1,23bis,23bis,NaN,FRAD094_3P_000255_01_0032,MAT_1813
2,34ter,34ter,NaN,FRAD094_3P_000255_01_0044,MAT_1813
3,46bis,46bis,NaN,FRAD094_3P_000255_01_0057,MAT_1813
4,107bis,107bis,NaN,FRAD094_3P_000255_01_0125,MAT_1813
...,...,...,...,...,...
148,1218,1218,NaN,FRAD094_3P_000264_01_0218,MAT_1848
149,1219,1219,NaN,FRAD094_3P_000264_01_0219,MAT_1848
150,1220,1220,NaN,FRAD094_3P_000264_01_0220,MAT_1848
151,1221,1221,NaN,FRAD094_3P_000264_01_0221,MAT_1848


In [353]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('source', srcuri)
g.bind('srctype', srctype)
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in folios_pages.iterrows():
    json = parse_record_id(row['Image'])
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

    subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['Num_Folio'])}")
    g.add((subject_uri, RDF.type, rico.RecordPart))
    g.add((subject_uri, cad.isSourceType, URIRef(srctype.Folio)))
    g.add((subject_uri, cad.hasNumFolio, Literal(row["Num_Folio_clean"],datatype=XSD.string)))
    g.add((subject_uri, rico.isOrWasConstituentOf,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))

    if not pd.isna(row['Alt_Num_CF']):
        g.add((subject_uri, cad.hasAlternativeNumFolio, Literal(int(row["Alt_Num_CF"]),datatype=XSD.string)))

print(g.serialize(format='turtle'))

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_MAT_B_NB_1813_107bis a rico:RecordPart ;
    cad:hasNumFolio "107bis"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0125_page .

source:94_Gentilly_MAT_B_NB_1813_11 a rico:RecordPart ;
    cad:hasNumFolio "11"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:94_Gentilly_MAT_B_NB_1813_122bis a rico:RecordPart ;
    cad:hasNumFolio "122bis"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0146_page .

source:94_Gentilly_MAT_B_NB_1813_124bis a rico:RecordPa

#### Création des folios issus de "Tiré de" et "Porté à" qui ne sont pas dans la colonne 'Num_Folios'

In [354]:
# Iterate over each row in the DataFrame
for index, row in folios.iterrows():
    if row['Tire_de_clean'] != 'EMPTY':
        ls = row['Tire_de_clean'].split(";")
        for l in ls:
            if any(num.isdigit() for num in l) and 'mission' not in l:
                MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

                subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(l)}")
                g.add((subject_uri, RDF.type, rico.RecordPart))
                g.add((subject_uri, cad.isSourceType, URIRef(srctype.Folio)))
                g.add((subject_uri, cad.hasNumFolio, Literal(l,datatype=XSD.string)))

In [355]:
# Iterate over each row in the DataFrame
for index, row in folios.iterrows():
    if row['Porte_a_clean'] != 'EMPTY':
        ls = row['Porte_a_clean'].split(";")
        for l in ls:
            #test if str has digit
            if any(num.isdigit() for num in l) and 'mission' not in l:
                MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]

                subject_uri = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(l)}")
                g.add((subject_uri, RDF.type, rico.RecordPart))
                g.add((subject_uri, cad.isSourceType, URIRef(srctype.Folio)))
                g.add((subject_uri, cad.hasNumFolio, Literal(l,datatype=XSD.string)))

In [356]:
print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_sources_folios.ttl", format='turtle')

@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:94_Gentilly_MAT_B_NB_1813_107bis a rico:RecordPart ;
    cad:hasNumFolio "107bis"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0125_page .

<http://data.ign.fr/id/source/94_Gentilly_MAT_B_NB_1813_107↑2↓> a rico:RecordPart ;
    cad:hasNumFolio "107↑2↓"^^xsd:string ;
    cad:isSourceType srctype:Folio .

source:94_Gentilly_MAT_B_NB_1813_11 a rico:RecordPart ;
    cad:hasNumFolio "11"^^xsd:string ;
    cad:isSourceType srctype:Folio ;
    rico:isOrWasConstituentOf source:94_Gentilly_FRAD094_3P_000255_01_0015_page .

source:94_Gentilly_MAT_B_NB_1813_122bis a rico:RecordPart ;
    cad:hasNumFolio "122bis"^^xsd:string ;
    cad:is

<Graph identifier=N9c6d0f495a2142d68afb68ea6e4b2aaa (<class 'rdflib.graph.Graph'>)>

### 1.3 Adresses

In [357]:
# Select the column as a new DataFrame
addresses = matrices[['registre','Lieu-dit_treated','Lieu-dit_type']].copy().drop_duplicates().reset_index(drop=True)
addresses

,registre,Lieu-dit_treated,Lieu-dit_type
0,MAT_1813,NaN,NaN
1,MAT_1813,Les Girantiers,District
2,MAT_1813,Rue Thiers,Thoroughfare
3,MAT_1813,Les Girantins,District
4,MAT_1813,Rue Thiers;10,Address
...,...,...,...
130,MAT_1848,Barrière d'Italie;8↑B↓,Address
131,MAT_1848,Barrière d'Italie;8↑A↓,Address
132,MAT_1848,Barrière d'Italie;6,Address
133,MAT_1848,Barrière d'Italie;4,Address


In [358]:
multipart_addresses_street = []
multipart_addresses_street_number = []
multipart_addresses_street_type = []
multipart_addresses_street_number_type = []


for index, row in addresses.iterrows():
    tag = str(row["Lieu-dit_treated"])
    if ';' in tag:
        add = tag.split(";")
        add_street_or_district = add[0]
        add_num_or_part = add[1]

        multipart_addresses_street.append(add_street_or_district)
        multipart_addresses_street_number.append(add_num_or_part)

        #test if digit
        if any(num.isdigit() for num in add_num_or_part):
            multipart_addresses_street_type.append('Thoroughfare')
            multipart_addresses_street_number_type.append('StreetNumber')
        else:
            multipart_addresses_street_type.append('District')
            multipart_addresses_street_number_type.append('Undefined')

    else:
        multipart_addresses_street.append('')
        multipart_addresses_street_number.append('')
        multipart_addresses_street_type.append('')
        multipart_addresses_street_number_type.append('')

addresses['part_street_district'] = multipart_addresses_street
addresses['part_street_number'] = multipart_addresses_street_number
addresses['part_street_district_type'] = multipart_addresses_street_type
addresses['part_street_number_type'] = multipart_addresses_street_number_type

#assign a distinct uuid for each group of rows with same values in Lieu-dit_treated and registre
addresses['address_uuid'] = [uuid.uuid4() for _ in range(len(addresses))]

addresses

,registre,Lieu-dit_treated,Lieu-dit_type,part_street_district,part_street_number,part_street_district_type,part_street_number_type,address_uuid
0,MAT_1813,NaN,NaN,,,,,4555a981-cfb9-4f62-8692-64bc88cbf13d
1,MAT_1813,Les Girantiers,District,,,,,ace7fa72-8ca6-4405-8bbd-f057a1fd776c
2,MAT_1813,Rue Thiers,Thoroughfare,,,,,419ec899-b25b-43ea-8ef5-0e1a8e2f9580
3,MAT_1813,Les Girantins,District,,,,,e8f44631-dc55-4b63-8f9b-7439652abbdc
4,MAT_1813,Rue Thiers;10,Address,Rue Thiers,10,Thoroughfare,StreetNumber,58c9ba97-f1ac-4372-ab3f-02144d66ca79
...,...,...,...,...,...,...,...,...
130,MAT_1848,Barrière d'Italie;8↑B↓,Address,Barrière d'Italie,8↑B↓,Thoroughfare,StreetNumber,23b6d823-ed85-42c2-b2a4-445703df2a7d
131,MAT_1848,Barrière d'Italie;8↑A↓,Address,Barrière d'Italie,8↑A↓,Thoroughfare,StreetNumber,a8609c6f-8fea-47bd-aeb4-a590099b66c7
132,MAT_1848,Barrière d'Italie;6,Address,Barrière d'Italie,6,Thoroughfare,StreetNumber,a65bbdad-835f-4e86-8295-3163f911542d
133,MAT_1848,Barrière d'Italie;4,Address,Barrière d'Italie,4,Thoroughfare,StreetNumber,d570189d-82c2-490a-bb40-abb845cbb915


In [359]:
import pandas as pd
import uuid

# Create a new column 'part_street_district_uuid'
addresses['part_street_district_uuid'] = None

# Create a dictionary to store the uuid for each unique part_street_district
uuid_dict = {}

# Iterate over the DataFrame
for i, row in addresses.iterrows():
    if row['part_street_district'] != '':
        # Check if the part_street_district value is equal to one of the Lieu-dit_treated values
        if row['part_street_district'] in addresses['Lieu-dit_treated'].values:
            # If yes, set the part_street_district_uuid to the uuid of the retrieved Lieu-dit_treated
            addresses.loc[i, 'part_street_district_uuid'] = addresses.loc[addresses['Lieu-dit_treated'] == row['part_street_district'], 'address_uuid'].values[0]
        else:
            # If no, check if the part_street_district value has other occurrences in the part_street_district column
            if row['part_street_district'] in addresses['part_street_district'].values:
                # If yes, check if the part_street_district value is already in the uuid_dict
                if row['part_street_district'] in uuid_dict:
                    # If yes, assign the same uuid
                    addresses.loc[i, 'part_street_district_uuid'] = uuid_dict[row['part_street_district']]
                else:
                    # If no, create a new uuid and add it to the uuid_dict
                    new_uuid = uuid.uuid4()
                    uuid_dict[row['part_street_district']] = new_uuid
                    addresses.loc[i, 'part_street_district_uuid'] = new_uuid
            else:
                # If no, create a new uuid
                addresses.loc[i, 'part_street_district_uuid'] = uuid.uuid4()

In [360]:
addresses

,registre,Lieu-dit_treated,Lieu-dit_type,part_street_district,part_street_number,part_street_district_type,part_street_number_type,address_uuid,part_street_district_uuid
0,MAT_1813,NaN,NaN,,,,,4555a981-cfb9-4f62-8692-64bc88cbf13d,None
1,MAT_1813,Les Girantiers,District,,,,,ace7fa72-8ca6-4405-8bbd-f057a1fd776c,None
2,MAT_1813,Rue Thiers,Thoroughfare,,,,,419ec899-b25b-43ea-8ef5-0e1a8e2f9580,None
3,MAT_1813,Les Girantins,District,,,,,e8f44631-dc55-4b63-8f9b-7439652abbdc,None
4,MAT_1813,Rue Thiers;10,Address,Rue Thiers,10,Thoroughfare,StreetNumber,58c9ba97-f1ac-4372-ab3f-02144d66ca79,419ec899-b25b-43ea-8ef5-0e1a8e2f9580
...,...,...,...,...,...,...,...,...,...
130,MAT_1848,Barrière d'Italie;8↑B↓,Address,Barrière d'Italie,8↑B↓,Thoroughfare,StreetNumber,23b6d823-ed85-42c2-b2a4-445703df2a7d,625a2df7-9ab9-458b-9c6a-9e75f50d2b45
131,MAT_1848,Barrière d'Italie;8↑A↓,Address,Barrière d'Italie,8↑A↓,Thoroughfare,StreetNumber,a8609c6f-8fea-47bd-aeb4-a590099b66c7,625a2df7-9ab9-458b-9c6a-9e75f50d2b45
132,MAT_1848,Barrière d'Italie;6,Address,Barrière d'Italie,6,Thoroughfare,StreetNumber,a65bbdad-835f-4e86-8295-3163f911542d,625a2df7-9ab9-458b-9c6a-9e75f50d2b45
133,MAT_1848,Barrière d'Italie;4,Address,Barrière d'Italie,4,Thoroughfare,StreetNumber,d570189d-82c2-490a-bb40-abb845cbb915,625a2df7-9ab9-458b-9c6a-9e75f50d2b45


In [361]:
from namespaces import *

# Create a new RDF graph
g = Graph()

landmarkuri = Namespace("http://data.ign.fr/id/landmark/")
g.bind('landmark', landmarkuri)
g.bind('source', srcuri)

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

g.bind("ltype", ltype)
g.bind("lrtype", lrtype)
g.bind("atype", atype)
g.bind('cad_ltype', cad_ltype)

for index, row in addresses.iterrows():
    add_uri = URIRef(landmarkuri + f"{row['address_uuid']}")
    g.add((add_uri, RDF.type, add.Landmark))
    if pd.notnull(addresses.loc[index, 'Lieu-dit_type']):
        if ';' in row['Lieu-dit_treated']:
            name = row['Lieu-dit_treated'].split(";")

            g.add((add_uri, add.isLandmarkType, URIRef(ltype + row['part_street_number_type'])))

            if any(num.isdigit() for num in name[1]):
                g.add((add_uri, SKOS.prefLabel, Literal(name[1] + ' (' + name[0] + ', ' + COMMUNE + ')', datatype=XSD.string)))
                g.add((add_uri, SKOS.altLabel, Literal(name[1] + ' ' + name[0], datatype=XSD.string)))
                g.add((add_uri,SKOS.hiddenLabel,Literal(name[1],datatype=XSD.string))) #Street number from transcription
                relationode = BNode()
                g.add((URIRef(relationode.n3()), add.isLandmarkRelationType, lrtype.Along))
            else:
                g.add((add_uri, SKOS.prefLabel, Literal(name[0] + ' (' + name[1] + ', ' + COMMUNE + ')', datatype=XSD.string)))
                g.add((add_uri, SKOS.altLabel, Literal(name[0], datatype=XSD.string)))
                relationode = BNode()
                g.add((URIRef(relationode.n3()), add.isLandmarkRelationType, lrtype.Undefined))

            g.add((URIRef(relationode.n3()), RDF.type, add.LandmarkRelation))
            g.add((URIRef(relationode.n3()), add.locatum, add_uri))
            g.add((URIRef(relationode.n3()), add.relatum, URIRef(landmarkuri + str(row['part_street_district_uuid']))))

            #Street or district relation with section
            sectionNode = BNode()
            g.add((URIRef(sectionNode.n3()), RDF.type, add.LandmarkRelation))
            g.add((URIRef(sectionNode.n3()), add.isLandmarkRelationType, lrtype.Within))
            g.add((URIRef(sectionNode.n3()), add.locatum, URIRef(landmarkuri + str(row['part_street_district_uuid']))))
            if row['registre'] != 'MAT_1848':
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + 'da6a5c2c-e86d-43bb-8950-7169bd0df60a'))) #Section D Cadastre 1848
            else:
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + '87d7c2f6-306b-45a1-a833-5e17821c3102'))) #Section B Cadastre 1811

        else:
            g.add((add_uri, add.isLandmarkType, URIRef(ltype + f"{row['Lieu-dit_type']}")))
            g.add((add_uri, SKOS.prefLabel, Literal(row['Lieu-dit_treated'] + ', ' + COMMUNE, datatype=XSD.string)))
            g.add((add_uri, SKOS.altLabel, Literal(row['Lieu-dit_treated'], datatype=XSD.string)))

            relationode = BNode()
            g.add((URIRef(relationode.n3()), add.isLandmarkRelationType, lrtype.Within))
            g.add((URIRef(relationode.n3()), add.locatum, add_uri))

            sectionNode = BNode()
            g.add((URIRef(sectionNode.n3()), RDF.type, add.LandmarkRelation))
            g.add((URIRef(sectionNode.n3()), add.isLandmarkRelationType, lrtype.Within))
            g.add((URIRef(sectionNode.n3()), add.locatum, add_uri))
            if row['registre'] != 'MAT_1848':
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + 'da6a5c2c-e86d-43bb-8950-7169bd0df60a'))) #Section D Cadastre 1848
            else:
                g.add((URIRef(sectionNode.n3()), add.relatum, URIRef(landmarkuri + '87d7c2f6-306b-45a1-a833-5e17821c3102'))) #Section B Cadastre 1811

        g.add((add_uri, cad.sourcedFrom, URIRef(srcuri + f'94_{COMMUNE}_{row["registre"]}')))

print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_landmarks_lieu_dit.ttl", format='turtle')

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix landmark: <http://data.ign.fr/id/landmark/> .
@prefix lrtype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkRelationType/> .
@prefix ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N0047bf2dc0cb47e995208050848e79f0> a add:LandmarkRelation ;
    add:isLandmarkRelationType lrtype:Within ;
    add:locatum landmark:419ec899-b25b-43ea-8ef5-0e1a8e2f9580 ;
    add:relatum landmark:87d7c2f6-306b-45a1-a833-5e17821c3102 .

<_:N00bb9b836dda40a186289920b3b4a8ca> a add:LandmarkRelation ;
    add:isLandmarkRelationType lrtype:Within ;
    add:locatum landmark:84b0b851-64e5-41df-95a2-78d97c014175 ;
    add:relatum landmark:87d7c2f6-306b-45a1-a833-5e17821c3102 .

<_:N01ad367e64b342a8a63

<Graph identifier=Na163db26684f4c3697fc618f49c8fe7e (<class 'rdflib.graph.Graph'>)>

### 1.5 Propriétaires

In [362]:
from pandasql import sqldf
import pandas as pd

In [363]:
matrices_ = matrices[['registre','Num_Folio','Type_CF','Groupe_CF','Image','UUID','Ordre_de_lecture']].copy()

In [364]:
#add column with rows index
owners_df['row_index'] = owners_df.index
owners_df2 = owners_df[['registre','type_folio','folio','groupe_cf','row_index']]
owners_df2.dtypes

registre      object
type_folio    object
folio         object
groupe_cf     object
row_index      int64
dtype: object

In [365]:
query = '''SELECT M.registre AS m_registre, M.Type_CF AS m_type_folio, M.Num_Folio AS m_num_folio, M.Groupe_CF AS m_groupe_cf, M.Image AS m_image, M.UUID AS m_uuid, m.Ordre_de_lecture AS m_row_num_in_cf, O.row_index AS o_row_index
        FROM matrices_ AS M
        LEFT JOIN owners_df2 AS O
        ON (M.Num_Folio = O.folio AND M.registre = O.registre AND M.Groupe_CF = O.groupe_cf AND M.Type_CF = O.type_folio)
        '''

testDF = sqldf(query)
testDF.to_csv(ROOT + 'test.csv',index=False)
print(sqldf(query))

    m_registre m_type_folio m_num_folio  m_groupe_cf                    m_image                                m_uuid  m_row_num_in_cf  o_row_index
0     MAT_1813         Bâti         108            1  FRAD094_3P_000255_01_0586  7c13b9ec-506c-4675-b691-335b828bbe11                1          0.0
1     MAT_1813         Bâti         114            1  FRAD094_3P_000255_01_0588  649a3bb0-7483-44c8-ad90-96bfc1808fdd                1          1.0
2     MAT_1813         Bâti          82            1  FRAD094_3P_000255_01_0579  bf640773-d6ff-4ba8-be3b-ffdd199dd8a4                1          2.0
3     MAT_1813         Bâti          82            1  FRAD094_3P_000255_01_0579  fe2c20fe-237f-4da7-8c7d-1ae33e5e1b96                2          2.0
4     MAT_1813         Bâti          64            1  FRAD094_3P_000255_01_0571  a8d13691-1a81-4684-9db7-98ee1fbc354d                1          3.0
..         ...          ...         ...          ...                        ...                                 

In [366]:
owners_matrices = pd.merge(owners_df, testDF, how='left', left_on=['row_index'], right_on=['o_row_index'])
#add a uuid to each distinct group of line with same values in registre, type_folio, folio and groupe_cf
owners_matrices.to_csv(ROOT + 'owners_matrices.csv',index=False)
owners_matrices

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription,row_index,m_registre,m_type_folio,m_num_folio,m_groupe_cf,m_image,m_uuid,m_row_num_in_cf,o_row_index
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],MAT_1813,Bâti,108,1,Legendre H↑re↓ de →Fontainebleau,0,MAT_1813,Bâti,108,1.0,FRAD094_3P_000255_01_0586,7c13b9ec-506c-4675-b691-335b828bbe11,1.0,0.0
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves,1,MAT_1813,Bâti,114,1.0,FRAD094_3P_000255_01_0588,649a3bb0-7483-44c8-ad90-96bfc1808fdd,1.0,1.0
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1.0,FRAD094_3P_000255_01_0579,bf640773-d6ff-4ba8-be3b-ffdd199dd8a4,1.0,2.0
3,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1.0,FRAD094_3P_000255_01_0579,fe2c20fe-237f-4da7-8c7d-1ae33e5e1b96,2.0,2.0
4,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Faipot', '...",[],MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...,3,MAT_1813,Bâti,64,1.0,FRAD094_3P_000255_01_0571,a8d13691-1a81-4684-9db7-98ee1fbc354d,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul,188,MAT_1848,Non Bâti,1221,1.0,FRAD094_3P_000264_01_0221,da62bcf1-3993-420b-8aac-e7cd1d1b4cda,6.0,188.0
414,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Nicolle', ...",[],MAT_1848,Non Bâti,1221,1,Nicolle J↑n↓ f↑ois↓ Paul,188,MAT_1848,Non Bâti,1221,1.0,FRAD094_3P_000264_01_0221,d0f7ae85-fd51-459b-a4b4-48a8335cf44a,7.0,188.0
415,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Lacroix', ...",[],MAT_1848,Non Bâti,1222,1,Lacroix J↑n↓ Louis,189,MAT_1848,Non Bâti,1222,1.0,FRAD094_3P_000264_01_0222,e0d84f8b-4b9b-4b54-92f0-e1d36d5e6e45,1.0,189.0
416,"{'registre': 'MAT_1848', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Lacroix', ...",[],MAT_1848,Non Bâti,1222,1,Lacroix J↑n↓ Louis,189,MAT_1848,Non Bâti,1222,1.0,FRAD094_3P_000264_01_0222,af70cad2-55f9-4163-8c6b-ece562802b66,2.0,189.0


In [367]:
#I want to create a dict with o_row_index as key and uuids as values
from datascroller import scroll

cf_uuid_dict = {}
sees = []
for index, row in owners_matrices.iterrows():
    if row['o_row_index'] not in sees:
        cf_uuid_dict[row['o_row_index']] = uuid.uuid4()
        sees.append(cf_uuid_dict[row['o_row_index']])

pd.set_option("display.max_rows", None)
display(owners_matrices)

,cell,owners,changes,registre,type_folio,folio,groupe_cf,transcription,row_index,m_registre,m_type_folio,m_num_folio,m_groupe_cf,m_image,m_uuid,m_row_num_in_cf,o_row_index
0,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Legendre',...",[],MAT_1813,Bâti,108,1,Legendre H↑re↓ de →Fontainebleau,0,MAT_1813,Bâti,108,1.0,FRAD094_3P_000255_01_0586,7c13b9ec-506c-4675-b691-335b828bbe11,1.0,0.0
1,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Louves', '...",[],MAT_1813,Bâti,114,1,Louves,1,MAT_1813,Bâti,114,1.0,FRAD094_3P_000255_01_0588,649a3bb0-7483-44c8-ad90-96bfc1808fdd,1.0,1.0
2,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1.0,FRAD094_3P_000255_01_0579,bf640773-d6ff-4ba8-be3b-ffdd199dd8a4,1.0,2.0
3,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Hardon', '...",[],MAT_1813,Bâti,82,1,"Hardon, Bourgeois ~~nourisseur~~→à Paris",2,MAT_1813,Bâti,82,1.0,FRAD094_3P_000255_01_0579,fe2c20fe-237f-4da7-8c7d-1ae33e5e1b96,2.0,2.0
4,"{'registre': 'MAT_1813', 'type_folio': 'Bâti',...","[{'owner-id': 1, 'owner-lastname': 'Faipot', '...",[],MAT_1813,Bâti,64,1,Faipot François→m↑d↓ de vin b↑re↓ de fontaineb...,3,MAT_1813,Bâti,64,1.0,FRAD094_3P_000255_01_0571,a8d13691-1a81-4684-9db7-98ee1fbc354d,1.0,3.0
5,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Bacot', 'o...",[],MAT_1813,Non Bâti,11,1,Bacot→(david) couverturier→à Paris,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Besson', '...","[{'change-order': 1, 'owner-before': 1, 'owner...",MAT_1813,Non Bâti,23bis,1,~~Besson Jard↑e↓→Fleuriste 1832→Lecoq Jean b↑t...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Benoist', ...",[],MAT_1813,Non Bâti,34ter,1,Benoist→md de vin en gros,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Bizouard',...",[],MAT_1813,Non Bâti,46bis,1,Bizouard,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"{'registre': 'MAT_1813', 'type_folio': 'Non Bâ...","[{'owner-id': 1, 'owner-lastname': 'Cornillon'...","[{'change-order': 1, 'owner-before': 1, 'owner...",MAT_1813,Non Bâti,107bis,1,~~Cornillon→vin primeur en caractère~~→1832→Co...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Créer les comptes fonciers
* Associer à chaque compte foncier ses propriétaires (ordonnés dans le temps)
* Associer à chaque compte foncier le landmark (état) qu'il mentionne

* ```o_row_index``` : id de l'article de mutation (et du compte foncier => 1..1)
donc
    * ArticleDeMutation : uuid_mutation
    * CompteFoncier : uuid
    * Owner : uuid_taxpayer = owner_id

* ```m_uuid``` : id de l'article de classement
donc
    * ArticleDeClassement : uuid

In [368]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('source', srcuri)
g.bind('taxpayer', owneruri)
g.bind('event', eventuri)

g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)
g.bind('cad_etype', cad_etype)
g.bind('ctype', ctype)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)
g.bind('rico', rico)
g.bind('add', add)
g.bind('cad', cad)

created_cf = []

for index, row in owners_matrices.iterrows():
    #Infos de l'image
    try:
        json = parse_record_id(row["m_image"])
        MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]
        #print(json)
        #UUID de l'article de classement (=uuid de la ligne)
        lineuuid_ = str(row['m_uuid'])

        #UUID du compte foncier
        cfuuid_ = str(cf_uuid_dict[row['o_row_index']])
        subject_uri = URIRef(srcuri + f"{cfuuid_}")

        #URI de l'article de classement
        articleclassementuri = URIRef(srcuri + f"{row['m_uuid']}_classement")
        g.add((articleclassementuri, RDF.type, rico.RecordPart))
        g.add((articleclassementuri, cad.isSourceType, URIRef(srctype.ArticleDeClassement)))
        g.add((articleclassementuri, DCTERMS.identifier, Literal(row['m_row_num_in_cf'],datatype=XSD.integer))) #Numéro d'ordre de l'article de classement dans le compte foncier

        #CF contient ArticleDeClassement
        g.add((subject_uri, rico.hasOrHadConstituent, articleclassementuri)) 

        if cfuuid_ not in created_cf:
            created_cf.append(cfuuid_)
            #URI du compte foncier
            g.add((subject_uri, RDF.type, rico.RecordPart))
            g.add((subject_uri, cad.isSourceType, URIRef(srctype.CompteFoncier)))
            g.add((subject_uri, rico.hasOrHadConstituent, URIRef(srcuri + f"{cfuuid_}_mutation"))) #CF contient ArticleDeMutation
            g.add((subject_uri, rico.isOrWasConstituentOf, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(row['m_num_folio'])}"))) #CF est contenu dans Folio
            g.add((subject_uri, DCTERMS.identifier, Literal(row['groupe_cf'],datatype=XSD.integer))) #Numéro d'ordre du compte foncier dans un folio

            #URI de l'article de mutation
            articlemutationuri = URIRef(srcuri + f"{cfuuid_}_mutation")
            g.add((articlemutationuri, RDF.type, rico.RecordPart))
            g.add((articlemutationuri, cad.isSourceType, URIRef(srctype.ArticleDeMutation)))

            ## Taxpayers contenu dans ArticleDeMutation
            ownersattribute = BNode()
            g.add((ownersattribute, RDF.type, add.Attribute))
            g.add((articlemutationuri, cad.hasCadastreAttribute, ownersattribute))
            g.add((ownersattribute, add.isAttributeType, URIRef(cad_atype.PlotTaxpayer)))

            owners_and_versions = {}
            for owner in row['owners']:
                ownersattributeversion = BNode()
                g.add((ownersattributeversion,RDF.type,add.AttributeVersion))
                g.add((ownersattribute, add.hasAttributeVersion, ownersattributeversion))
                owneruriinstance = URIRef(owneruri + f"{cfuuid_}_taxpayer_{owner['owner-id']}")
                g.add((owneruriinstance, RDF.type, cad.Taxpayer))
                g.add((ownersattributeversion, cad.hasTaxpayer, owneruriinstance))
                olabel = ''
                #test if json has a certain key
                if 'owner-lastname' in owner:
                    g.add((owneruriinstance, cad.taxpayerLabel, Literal(owner['owner-lastname'],datatype=XSD.string)))
                    olabel += owner['owner-lastname']
                if 'owner-firstname' in owner:
                    g.add((owneruriinstance, cad.taxpayerFirstName, Literal(owner['owner-firstname'],datatype=XSD.string)))
                    olabel += ' ' + owner['owner-firstname']
                if 'owner-status' in owner:
                    g.add((owneruriinstance, cad.taxpayerStatus, Literal(owner['owner-status'],datatype=XSD.string)))
                    olabel += ' (' + owner['owner-status'] + ')'
                if 'owner-activity' in owner:
                    g.add((owneruriinstance, cad.taxpayerActivity, Literal(owner['owner-activity'],datatype=XSD.string)))
                if 'owner-address' in owner:
                    g.add((owneruriinstance, cad.taxpayerAddress, Literal(owner['owner-address'],datatype=XSD.string)))
                #Create owner label
                g.add((owneruriinstance, RDFS.label, Literal(olabel,datatype=XSD.string)))
                g.add((owneruriinstance,cad.fromSource,articlemutationuri))
                owners_and_versions[owner['owner-id']] = ownersattributeversion

            if len(row) > 0:
                for change in row['changes']:
                    changenode = BNode()
                    g.add((URIRef(changenode.n3()), RDF.type, add.Change))
                    g.add((URIRef(changenode.n3()), add.appliedTo, ownersattribute))
                    g.add((URIRef(changenode.n3()), add.outdates, URIRef(owners_and_versions[change['owner-before']])))
                    g.add((URIRef(changenode.n3()), add.makesEffective, URIRef(owners_and_versions[change['owner-after']])))
                    g.add((URIRef(changenode.n3()),add.isChangeType,ctype.AttributeVersionTransition))
                    event_uuid = uuid.uuid4()
                    event_uri = URIRef(eventuri + f"{event_uuid}")
                    g.add((URIRef(changenode.n3()), add.dependsOn, event_uri))
                    if 'date' in change:
                        g.add((event_uri, RDF.type, add.Event))
                        time_ = BNode()
                        g.add((event_uri, add.hasTime, time_))
                        g.add((event_uri, cad.isEventType, cad_etype.TaxpayerMutation))
                        g.add((time_, RDF.type, add.TimeInstant))
                        g.add((time_, add.timeCalendar, time.Gregorian))
                        g.add((time_, add.Precision, time.Year))
                        g.add((time_, add.timeStamp, Literal(change['date'],datatype=XSD.dateTimeStamp)))
            #Create initial change
            initchangenode = BNode()
            g.add((URIRef(initchangenode.n3()), RDF.type, add.Change))
            g.add((URIRef(initchangenode.n3()), add.appliedTo, ownersattribute))
            g.add((URIRef(initchangenode.n3()), add.makesEffective, URIRef(owners_and_versions[1])))
            g.add((URIRef(initchangenode.n3()), add.isChangeType, ctype.AttributeVersionAppearance))
            event_uuid = uuid.uuid4()
            event_uri = URIRef(eventuri + f"{event_uuid}")
            g.add((URIRef(initchangenode.n3()), add.dependsOn, event_uri))

            #Create final change
            finalchangenode = BNode()
            g.add((URIRef(finalchangenode.n3()), RDF.type, add.Change))
            g.add((URIRef(finalchangenode.n3()), add.appliedTo, ownersattribute))
            g.add((URIRef(finalchangenode.n3()), add.outdates, URIRef(owners_and_versions[len(owners_and_versions)])))
            g.add((URIRef(finalchangenode.n3()), add.isChangeType, ctype.AttributeVersionDisappearance))
            event_uuid = uuid.uuid4()
            event_uri = URIRef(eventuri + f"{event_uuid}")
            g.add((URIRef(finalchangenode.n3()), add.dependsOn, event_uri))
    except:
        print(row)

print(g.serialize(format='turtle'))
g.serialize(destination=f"{OUTPUT_FOLDER_PATH}/{COMMUNE}_owners_cf_clas_mut.ttl", format='turtle')

cell               {'registre': 'MAT_1813', 'type_folio': 'Non Bâ...
owners             [{'owner-id': 1, 'owner-lastname': 'Bacot', 'o...
changes                                                           []
registre                                                    MAT_1813
type_folio                                                  Non Bâti
folio                                                             11
groupe_cf                                                          1
transcription                     Bacot→(david) couverturier→à Paris
row_index                                                          4
m_registre                                                       NaN
m_type_folio                                                     NaN
m_num_folio                                                      NaN
m_groupe_cf                                                      NaN
m_image                                                          NaN
m_uuid                            

@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://data.ign.fr/def/cadastre#> .
@prefix cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/> .
@prefix cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/> .
@prefix ctype: <http://rdf.geohistoricaldata.org/id/codes/address/changeType/> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix event: <http://data.ign.fr/id/event/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://data.ign.fr/id/source/> .
@prefix srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/> .
@prefix taxpayer: <http://data.ign.fr/id/taxpayer/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<_:N00c2e89fe2124292b9e783ee33b20bdd> a add:Change ;
    add:appliedTo _:N600489b82deb42d19b4b2061d1a897ec ;
    add:dependsOn event:8782d855-2ccb-4301-9e68-94242622f699 ;
  

<Graph identifier=N9061d98067cb4fce8ee9dc8af2bf128c (<class 'rdflib.graph.Graph'>)>

### 1.X Création des états de parcelles

In [369]:
matrices

,ID,UUID,Type_CF,Num_Folio,Alt_Num_CF,Groupe_CF,Ordre_de_lecture,Voie,Num_Voie,Image,...,Porté à_treated,Ligne barrée ?,CF rayé ?,Spécification,Commentaire,Cote liée,registre,Num_Folio_clean,Tire_de_clean,Porte_a_clean
0,1,7c13b9ec-506c-4675-b691-335b828bbe11,Bâti,108,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0586,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,108,EMPTY,EMPTY
1,2,649a3bb0-7483-44c8-ad90-96bfc1808fdd,Bâti,114,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0588,...,82,Oui,Oui,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,114,EMPTY,82
2,3,bf640773-d6ff-4ba8-be3b-ffdd199dd8a4,Bâti,82,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,82,114,EMPTY
3,4,fe2c20fe-237f-4da7-8c7d-1ae33e5e1b96,Bâti,82,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0579,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,82,EMPTY,EMPTY
4,5,a8d13691-1a81-4684-9db7-98ee1fbc354d,Bâti,64,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0571,...,NaN,Non,Non,Matrice des propriétés bâties ne sont plus mis...,NaN,NaN,MAT_1813,64,EMPTY,EMPTY
5,6,7008c4af-2b07-4630-8667-9c8783aced10,Non bâti,11,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0015,...,156,Non,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,156
6,7,ef348452-7650-41a5-ac99-61d2490800b5,Non bâti,11,NaN,1,2,NaN,NaN,FRAD094_3P_000255_01_0015,...,156,Non,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,156
7,8,6e7b862a-4f66-461c-96d4-db9f2dc76e56,Non bâti,11,NaN,1,3,NaN,NaN,FRAD094_3P_000255_01_0015,...,192bis,Oui,Oui,NaN,NaN,NaN,MAT_1813,11,EMPTY,192bis
8,9,96e406d5-0647-4889-954d-b821f15ba128,Non bâti,23bis,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0032,...,280bis,Oui,Oui,NaN,NaN,NaN,MAT_1813,23bis,constructionnouvellesv,280bis
9,10,3076b547-3147-4247-b503-74e9ead7af45,Non bâti,34ter,NaN,1,1,NaN,NaN,FRAD094_3P_000255_01_0044,...,NaN,Non,Oui,NaN,NaN,NaN,MAT_1813,34ter,46ter,EMPTY


In [370]:
matrices.columns

Index(['ID', 'UUID', 'Type_CF', 'Num_Folio', 'Alt_Num_CF', 'Groupe_CF',
       'Ordre_de_lecture', 'Voie', 'Num_Voie', 'Image', 'Section_clean',
       'Parcelle_clean', 'Lieu-dit_transcript', 'Lieu-dit_clean',
       'Lieu-dit_treated', 'Lieu-dit_type', 'Propriétaires_transcript',
       'Nature_transcript', 'Nature_clean', 'Nature_treated', 'Date entrée',
       'Date entrée_treated', 'Date sortie', 'Date sortie_treated', 'Tiré de',
       'Tiré de_treated', 'Porté à', 'Porté à_treated', 'Ligne barrée ?',
       'CF rayé ?', 'Spécification', 'Commentaire', 'Cote liée', 'registre',
       'Num_Folio_clean', 'Tire_de_clean', 'Porte_a_clean'],
      dtype='object')

In [371]:
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('landmark', landmarkuri)
g.bind('owner', owneruri)
g.bind('source', srcuri)
g.bind('cad_ltype', cad_ltype)
g.bind('cad_atype', cad_atype)
g.bind('cad_etype', cad_etype)
g.bind('ctype', ctype)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)

g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)

# Iterate over each row in the DataFrame
for index, row in matrices.iterrows():
    #Infos de l'image
    json = parse_record_id(row["Image"])
    #Infos de la matrice et du plan
    MATRICE_ID = matrices_metada[row['registre']]["MATRICE_ID"]
    PLAN = matrices_metada[row['registre']]["PLAN"]
    #Article de classement correspondant à la ligne
    articleclassementuri = URIRef(srcuri + f"{row['UUID']}_classement")

    #Création de l'état des landmarks de type Plot
    subject_uri = URIRef(landmarkuri + f"{row['UUID']}")
    g.add((subject_uri, RDF.type, add.Landmark))
    g.add((subject_uri, add.isLandmarkType, cad_ltype.Plot))
    g.add((subject_uri, DCTERMS.identifier, Literal(row['Section_clean'] + '-' + str(row['Parcelle_clean']), datatype=XSD.string)))

    #Article de classement et folios précédents/suivants
    plotMentionAttr = BNode()
    g.add((subject_uri, add.hasAttribute,plotMentionAttr))
    g.add((plotMentionAttr, RDF.type, add.Attribute))
    g.add((plotMentionAttr, add.isAttributeType, cad_atype.PlotMention))
    plotMentionAttrVersion = BNode()
    g.add((plotMentionAttr, add.hasAttributeVersion, plotMentionAttrVersion))
    g.add((plotMentionAttrVersion, RDF.type, add.AttributeVersion))
    
    ## Article de classement
    g.add((plotMentionAttrVersion, cad.isMentionnedIn, articleclassementuri))
    ## Folio.s ou valeur.s spéciale.s précédent.s (Tiré de)
    tire_de = str(row['Tiré de_treated']).split(';')
    print(tire_de)
    for f in tire_de:
        if any(char.isdigit() for char in str(f)) and 'mission' not in f:
            g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(f)}")))
        elif 'mission' in f:
            g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(cad_spval + f"OmissionSV")))
        elif f != 'nan':
            print(f)
            g.add((plotMentionAttrVersion, cad.takenFrom, URIRef(cad_spval + f"{str(f)}")))
    ## Folio.s ou valeur.s spéciale.s  suivant.s (Porté à)
    porte_a = str(row['Porté à_treated']).split(';')
    for f in porte_a:
        if any(char.isdigit() for char in str(f)) and 'mission' not in f:
            g.add((plotMentionAttrVersion, cad.passedTo, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}_{str(f)}")))
        elif f != 'nan':
            print(f)
            g.add((plotMentionAttrVersion, cad.passedTo, URIRef(cad_spval + f"{str(f)}")))

    #Time
    if not pd.isnull(row['Date entrée']) or not pd.isnull(['Date sortie']):
        hastime = BNode()
        g.add((subject_uri, add.hasTime, hastime))
        g.add((hastime, RDF.type, add.TimeInterval))
        
        if not pd.isna(row['Date entrée']) and not pd.isnull(row['Date entrée']):
            hasbeginning = BNode()
            g.add((hastime, add.hasBeginning, hasbeginning))
            g.add((hasbeginning,RDF.type, add.TimeInstant))
            g.add((hasbeginning, add.timeCalendar, time.Gregorian))
            g.add((hasbeginning, add.timePrecision, time.Year))
            g.add((hasbeginning, add.timeStamp, Literal(row['Date entrée'], datatype=XSD.dateTimeStamp)))
        #else:
            #g.add((hastime, add.hasBeginning, Literal(MATRICE_START, datatype=XSD.date)))#date d'ouverture de la matrice
        if not pd.isnull(row['Date sortie']) and row['Date sortie'] != 'nan':
            hasend = BNode()
            g.add((hastime, add.hasBeginning, hasend))
            g.add((hasend,RDF.type, add.TimeInstant))
            g.add((hasend, add.timeCalendar, time.Gregorian))
            g.add((hasend, add.timePrecision, time.Year))
            g.add((hasend, add.timeStamp, Literal(row['Date entrée'], datatype=XSD.dateTimeStamp)))

    ## Intervale

    #Source
    rowSource = BNode()
    g.add((subject_uri, cad.sourcedFrom, rowSource))
    g.add((rowSource, RDF.type, fpo.SourceCitation))
    g.add((rowSource, fpo.fromSource, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{MATRICE_ID}")))
    g.add((rowSource, rico.isComponentOfTransitive, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    g.add((rowSource, PROV.wasGeneratedBy, URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0002")))
    g.add((URIRef(f"http://data.ign.fr/id/codes/cadastre/activity/0002"), PROV.used, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}")))
    g.add((rowSource, rico.isOrWasDigitalInstanciation,URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")))
    g.add((rowSource, cad.lineOrderInArea, Literal(row['Ordre_de_lecture'], datatype=XSD.integer)))

    #Create recordpart
    recordparturi = URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_{lineuuid_}_area")
    g.add((recordparturi, RDF.type, rico.RecordPart))
    g.add((recordparturi, rico.isOrWasIncludedIn, URIRef(srcuri + f"{json['departement']}_{COMMUNE}_{row['Image']}_page")))
    
    #Address
    if row['Lieu-dit_treated'] != 'nan':
        plotaddress = BNode()
        g.add((subject_uri, add.hasAttribute, plotaddress))
        g.add((plotaddress, add.isAttributeType, cad_atype.PlotAddress))
        plotaddressversion = BNode()
        g.add((plotaddress, add.hasAttributeVersion, plotaddressversion))
        g.add((plotaddressversion, RDF.type, add.LandmarkRelation))
        g.add((plotaddressversion, add.isLandmarkRelationType, lrtype.Undefined))
        g.add((plotaddressversion, add.locatum, subject_uri))
        #g.add((plotaddressversion, add.relatum, URIRef(landmarkuri + str(row['address_uuid']))))


    #Nature
    if not pd.isnull(row['Nature_treated']):
        nature = BNode()
        g.add((subject_uri, add.hasNature, nature))
        g.add((nature, RDF.type, add.Nature))
        g.add((nature, RDFS.label, Literal(row['Nature_treated'], datatype=XSD.string)))

print(g.serialize(format='turtle'))

['nan']
['nan']
['114']
['nan']
['nan']
['nan']
['nan']
['nan']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['46ter']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['238']
['268']
['274²']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['228↑ter↓']
['293']
['440']
['11']
['11']
['nan']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['440']
['11']
['236']
['211bis']
['211bis']
['211']
['211']
['211']
['260']
['260']
['nan']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['440']
['nan']
['295']
['440']
VoiePubliqueSV
['236ter']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['nan']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['440']
['nan']
['nan']
['nan']
['nan']
['281']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
DoubleEmploiSV
['23bis']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['nan']
['ConstructionNouvelleSV']
ConstructionNouvelleSV
['nan']
['